# Packages

In [ ]:
import cv2

from google.colab import files
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from IPython.display import clear_output, HTML
clear_output()

# Prep

In [ ]:
path = '/usr/local/share/jupyter/nbextensions/google.colab'
!cp -r {path}/* .
!rm -r {path}
!ln -s /content {path}
# change base tag

def change_base_url():
  eval_js("""
  var base = document.createElement('base')
  base.href = 'https://localhost:8080/nbextensions/google.colab/'
  document.head.prepend(base)
  """)
# make it run automatically in every cell
get_ipython().events.register('pre_run_cell', change_base_url)

fpath_img1 = 'pose_sample2.jpeg'
!gdown 1akyrQ4k_2weAN1rFOZ9xJ-ReUhqM3anb -O "$fpath_img1"

fpath_img2 = 'od_sample2.jpg'
!gdown 19UxDH_g3EHwEnz1WvLSlXe6D8ypgIwNC -O "$fpath_img2"

clear_output()

# 動態

In [ ]:
#@title WebCam Image
html_code3 = HTML('''
<!-- 相關依賴套件載入 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-core"></script>
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-converter"></script>

<!-- 本章節採用tfjs，因此載入TF.js backend -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-backend-webgl"></script>

<!-- 載入手勢辨識模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/pose-detection"></script>

<!-- 用於呈現分析結果canvas -->
<canvas id='canvas1' height="0" width="0"></canvas>

<script>
// 宣告canvas 物件
const canvas = document.getElementById('canvas1');
// 宣告canvas 繪圖物件
var ctx = canvas.getContext('2d')

// webcam 攝影
async function takePhoto() {
  await tf.setBackend('webgpu')
  // 讀取模型
  const detectorConfig = {modelType: poseDetection.movenet.modelType.SINGLEPOSE_LIGHTNING};
  const detector = await poseDetection.createDetector(poseDetection.SupportedModels.MoveNet, detectorConfig);

  const div = document.createElement('div'); // 空行
  const capture = document.createElement('button'); // 攝影按鈕
  capture.textContent = 'Capture'; // 攝影按鈕文字
  div.appendChild(capture);

  const video = document.createElement('video'); // 宣告影片物件
  video.style.display = 'block';
  const stream = await navigator.mediaDevices.getUserMedia({video: true}); // 宣告串流物件

  document.body.appendChild(div);
  div.appendChild(video);
  video.srcObject = stream; // 使影片物件顯示串流內容
  await video.play();

  // 調整Video Element 大小
  google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

  // 等待點擊攝影按鈕
  await new Promise((resolve) => capture.onclick = resolve);

  // canvas 繪製底圖
  canvas.width = video.videoWidth;
  canvas.height = video.videoHeight;
  ctx.drawImage(video, 0, 0);

  // 停止 webcamera
  stream.getVideoTracks()[0].stop();
  div.remove();

  // 模型推論分析
  const poses = await detector.estimatePoses(canvas1);
  console.log(poses)

  // 人體關節點註解文字顏色
  ctx.strokeStyle = "red";
  // 人體關節點註解文字大小&字體
  ctx.font = "20px Arial";
  // 繪製模型分析結果
  keypoints = poses[0].keypoints
  for (var i = 0; i < keypoints.length; i++) {
      console.log(keypoints[i]);

      // 關節點pixel-wise位置
      x = keypoints[i].x
      y = keypoints[i].y

      // 關節點類別
      name = keypoints[i].name

      // 圈選關節點估測位置
      ctx.beginPath();
      ctx.arc(x, y, 20, 0, 2 * Math.PI);
      ctx.stroke();

      // 繪製關節點文字註解
      ctx.fillText(name, x+30, y);
  }
}

takePhoto()
</script>
''')

html_code3

In [ ]:
#@title WebCam Video
html_code4 = HTML('''
<head>
  <!-- 相關依賴套件載入 -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-core"></script>
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-converter"></script>

  <!-- 本章節採用tfjs，因此載入TF.js backend -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-backend-webgl"></script>

  <!-- 載入手勢辨識模型 -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/pose-detection"></script>

  <!-- 隱藏html video -->
  <style>
      video{
          visibility: hidden;
      }
  </style>
</head>

<!-- 呈現結果的canvas -->
<canvas id="detect_result"></canvas>
<!-- 串流video -->
<video autoplay playsinline muted id="webcam" ></video>

<script>
async function app() {
  await tf.setBackend('webgpu')
  // 讀取模型
  const detectorConfig = {modelType: poseDetection.movenet.modelType.SINGLEPOSE_LIGHTNING};
  const detector = await poseDetection.createDetector(poseDetection.SupportedModels.MoveNet, detectorConfig);

  const webcamElement = document.getElementById('webcam'); // 宣告影片物件

  // 宣告canvas物件
  const canvas = document.getElementById('detect_result');
  const ctx = canvas.getContext('2d');

  let showResult = async function () {
    // 模型推論分析
    const poses = await detector.estimatePoses(canvas);
    console.log(poses)

    // canvas 繪製底圖
    canvas.width = webcamElement.videoWidth;
    canvas.height = webcamElement.videoHeight;
    ctx.drawImage(webcamElement, 0, 0);

    // 人體關節點註解文字顏色
    ctx.strokeStyle = "red";
    // 人體關節點註解文字大小&字體
    ctx.font = "20px Arial";
    // 繪製模型分析結果
    if (poses.length>0){
      keypoints = poses[0].keypoints
      for (var i = 0; i < keypoints.length; i++) {
          console.log(keypoints[i]);

          // 關節點pixel-wise位置
          x = keypoints[i].x
          y = keypoints[i].y

          // 關節點類別
          name = keypoints[i].name

          // 圈選關節點估測位置
          ctx.beginPath();
          ctx.arc(x, y, 20, 0, 2 * Math.PI);
          ctx.stroke();

          // 繪製關節點文字註解
          ctx.fillText(name, x+30, y);
      }
    }

    // 設定每偵顯示秒數
    setTimeout(function () {
        showResult();
    }, 300);
  }
  let setupWebcam = function () {
      return new Promise((resolve, reject) => {
          const navigatorAny = navigator;
          navigator.getUserMedia = navigator.getUserMedia ||
              navigatorAny.webkitGetUserMedia || navigatorAny.mozGetUserMedia ||
              navigatorAny.msGetUserMedia;
          if (navigator.getUserMedia) {
              navigator.getUserMedia({
                      video: true
                  },
                  (stream) => {
                      webcamElement.srcObject = stream;
                      webcamElement.addEventListener('loadeddata', () => resolve(),
                          false);
                  },
                  (err) => reject(err));
          } else {
              reject("getUserMedia failed");
          }
      });
  }
  setupWebcam().then(
      () => {
          showResult();
      },
      (err) => {
          console.log(err);
      }
  )
}
app();
</script>
''')

html_code4